Домашнее задание по лекции "Подготовка данных (Data preprocessing)", Масляков Глеб.

In [1]:
import numpy as np
import pandas as pd
from time import time

Задача: улучшить код со слайдов (если возможно).

#### Пример $1$: (слайд $11$)
перевести денежные суммы формата "string(sum\$)" в целые числа "integer(sum)". 

Генерация случайных данных.

Миллион записей от $0$ до $100000\$ $.

In [2]:
df = pd.DataFrame({'price($)': [str(i) + '$'  for i in np.random.choice(a = range(int(1e5)), size = int(1e6))]})

In [3]:
df.shape

(1000000, 1)

In [4]:
df.head()

,price($)
0,85687$
1,99667$
2,30080$
3,81602$
4,11177$


Время работы примера из лекции.

In [5]:
%%time
#Здесь начинается код из лекции
df['.'] = df['price($)'].apply(lambda x: int(x[:-1]))
#Здесь он заканчивается

CPU times: user 714 ms, sys: 35.1 ms, total: 749 ms
Wall time: 749 ms


In [6]:
df.head()

,price($),.
0,85687$,85687
1,99667$,99667
2,30080$,30080
3,81602$,81602
4,11177$,11177


Удаление получившихся результатов

In [7]:
del df['.']

Оптимизированная версия

In [8]:
%%time
df['.'] = df['price($)'].apply(lambda x: x.replace('$', '')).astype(int)

CPU times: user 540 ms, sys: 26.4 ms, total: 566 ms
Wall time: 566 ms


###########

Данный код ещё и лучше тем, что может применяться в ситуации, когда значок доллара стоит в начале строки.

###########

#### На следующем слайде возникла задача перекодировки строковых категориальных признаков в числа.

Проблема в строчке, где надо было перекодировать слова 'yes' и 'no' в числа $1$ и $0$. Её можно реализовать эффективнее.

Генерация данных

$10$ миллионов записей вида ['yes/no', 'warm/cool/cold']

In [9]:
f1 = lambda x: 'yes' if x == 1 else 'no'
def f2(x):
    if x == 0:
        return 'cool'
    elif x == 1:
        return 'cold'
    else:
        return 'warm'

In [19]:
df = pd.DataFrame({'ans': [f1(i) for i in np.random.choice(2, 10000000)],
                   'weather': [f2(i) for i in np.random.choice(3, 10000000)]})

In [20]:
df.head()

,ans,weather
0,no,cool
1,yes,cool
2,yes,cool
3,yes,cold
4,no,cold


Пример из лекции

In [21]:
%%time
dct = {'yes': 1, 'no': 0} 
df['ans_coded'] = df['ans'].map(dct)

CPU times: user 528 ms, sys: 15.8 ms, total: 544 ms
Wall time: 542 ms


In [22]:
df.head()

,ans,weather,ans_coded
0,no,cool,0
1,yes,cool,1
2,yes,cool,1
3,yes,cold,1
4,no,cold,0


Удаляем результат

In [23]:
del df['ans_coded']

Оптимизированная версия. Используем встроенную функцию factorize.

In [24]:
%%time
df['ans_coded'] = df['ans'].factorize(sort=True)[0]

CPU times: user 406 ms, sys: 47.8 ms, total: 454 ms
Wall time: 456 ms


In [25]:
df.head()

,ans,weather,ans_coded
0,no,cool,0
1,yes,cool,1
2,yes,cool,1
3,yes,cold,1
4,no,cold,0


In [26]:
del df

#### Корректировка значений (слайд $13$)

На следующем слайде предлагается извлечь нижнее и верхнее давление из записи вида string(v.d./n.d.)

Генерация данных

Миллион записей. Верхнее от $0$ до $200$, нижнее от $0$ до $150$

In [29]:
pressure = [str(np.random.choice(200)) + '/' + str(np.random.choice(150)) for _ in range(1000000)]
df = pd.DataFrame(pressure, columns = ['давление'])

In [30]:
df.head()

,давление
0,164/28
1,182/85
2,147/49
3,44/87
4,85/60


Пример из лекции

In [31]:
%%time
tmp = df['давление'].str.split('/')
df['в.давл.'] = tmp.apply(lambda x: x[0]) 
df['н.давл.'] = tmp.apply(lambda x: x[1])

CPU times: user 1.55 s, sys: 107 ms, total: 1.66 s
Wall time: 1.66 s


In [32]:
df.head()

,давление,в.давл.,н.давл.
0,164/28,164,28
1,182/85,182,85
2,147/49,147,49
3,44/87,44,87
4,85/60,85,60


Удаляем результаты

In [33]:
del df['в.давл.']
del df['н.давл.']

Оптимизированная версия. Не делаем два apply, а сразу скармливаем pd.DataFrame предварительно переведя в формат list

In [38]:
%%time
df[['в.давл.', 'н.давл.']] = pd.DataFrame(df['давление'].str.split('/', 1).tolist(), columns = ['Давление_в','Давление_н'])

CPU times: user 1.19 s, sys: 83.7 ms, total: 1.28 s
Wall time: 1.27 s


In [39]:
del df

#### Заполняем пропуски средними значениями (слайд 19)

Необходимо заполнить Nanы средними значениями. В лекции предложены три варианта заполнения: средним по всей выборке; средним по обучающей выборке; пропуски в обучающей выборке &mdash; средним по обучающей выборке, пропуски в тестовой выборке &mdash; средним по тестовой выборке.

Претензии по коду есть именно по третьему варианту.

Генерация данных

$6,000,000$ записей. Значения площадей от $0$ до $200$. Доля трейна и теста &mdash; $50\%$ (как на слайде). Доля Nanов &mdash; $\frac{1}{3}$ (тоже как на слайде).

In [40]:
df = pd.DataFrame(np.random.choice(200, size = (6000000, 4)), columns = ['площадь', 'площадь 1', 'площадь 2', 'площадь 3'])
x = np.array(['train'] * 3000000 + ['test'] * 3000000)
np.random.shuffle(x)
df['data'] = x
ind = np.arange(6000000)
np.random.shuffle(ind)
df.loc[ind[:2000000], 'площадь'] = np.nan

In [41]:
df.head(10)

,площадь,площадь 1,площадь 2,площадь 3,data
0,41.0,121,190,22,train
1,148.0,127,167,87,train
2,152.0,128,35,194,train
3,NaN,182,195,32,train
4,169.0,113,66,90,train
5,40.0,97,111,14,train
6,111.0,34,116,150,train
7,177.0,132,38,195,train
8,NaN,102,66,188,test
9,148.0,96,78,42,train


Пример из лекции. Очень объёмный код.

In [42]:
%%time
df.loc[df['data'] == 'train', 'площадь'] = df[df['data'] == 'train']['площадь'].fillna(df[df['data'] == 'train']['площадь'].mean())
df.loc[df['data'] == 'test', 'площадь'] = df[df['data'] == 'test']['площадь'].fillna(df[df['data'] == 'test']['площадь'].mean())

CPU times: user 3.93 s, sys: 170 ms, total: 4.1 s
Wall time: 4.12 s


In [43]:
df.head(10)

,площадь,площадь 1,площадь 2,площадь 3,data
0,41.000000,121,190,22,train
1,148.000000,127,167,87,train
2,152.000000,128,35,194,train
3,99.504992,182,195,32,train
4,169.000000,113,66,90,train
5,40.000000,97,111,14,train
6,111.000000,34,116,150,train
7,177.000000,132,38,195,train
8,99.458821,102,66,188,test
9,148.000000,96,78,42,train


Возвращаем Nanы на место

In [44]:
df.loc[ind[:2000000], 'площадь'] = np.nan

Напрашивается сделать группировку по столбцу "data". Также можно воспользоваться функцией transform.

In [46]:
%%time
df['площадь'] = df.groupby("data")['площадь'].transform(lambda x: x.fillna(x.mean()))

CPU times: user 1.61 s, sys: 298 ms, total: 1.9 s
Wall time: 2.02 s


В одну строчку. В два раза быстрее.

In [48]:
df.head(10)

,площадь,площадь 1,площадь 2,площадь 3,data
0,41.000000,121,190,22,train
1,148.000000,127,167,87,train
2,152.000000,128,35,194,train
3,99.504992,182,195,32,train
4,169.000000,113,66,90,train
5,40.000000,97,111,14,train
6,111.000000,34,116,150,train
7,177.000000,132,38,195,train
8,99.458821,102,66,188,test
9,148.000000,96,78,42,train


Можно попробовать ещё сильнее ускорить.

Возвращаем Nanы

In [55]:
df.loc[ind[:2000000], 'площадь'] = np.nan

Супер оптимизация

In [56]:
%%time
df.loc[df['площадь'].isnull(), 'площадь'] = df.groupby('data')['площадь'].transform('mean')

CPU times: user 507 ms, sys: 38.5 ms, total: 545 ms
Wall time: 546 ms


In [57]:
df.head(10)

,площадь,площадь 1,площадь 2,площадь 3,data
0,41.000000,121,190,22,train
1,148.000000,127,167,87,train
2,152.000000,128,35,194,train
3,99.504992,182,195,32,train
4,169.000000,113,66,90,train
5,40.000000,97,111,14,train
6,111.000000,34,116,150,train
7,177.000000,132,38,195,train
8,99.458821,102,66,188,test
9,148.000000,96,78,42,train


Ещё в несколько раз быстрее.